In [1]:
import json
import pandas as pd
import sqlite3

# Connect to the SQLite database
connection = sqlite3.connect("")
cursor = connection.cursor()

# Create tables
cursor.execute("""CREATE TABLE type (
                    id INTEGER PRIMARY KEY,
                    name TEXT
                )""")

cursor.execute("""CREATE TABLE property (
                    id INTEGER PRIMARY KEY,
                    name TEXT,
                    domain INTEGER,
                    range INTEGER,
                    required BOOLEAN,
                    multi TEXT,
                    FOREIGN KEY (domain) REFERENCES property_type(id),
                    FOREIGN KEY (range) REFERENCES property_type(id)
                )""")

cursor.execute("""CREATE TABLE property_type (
                    id INTEGER PRIMARY KEY,
                    type_id INTEGER,
                    property_id INTEGER,
                    FOREIGN KEY (type_id) REFERENCES type(id),
                    FOREIGN KEY (property_id) REFERENCES property(id)
                )""")

connection.commit()

# Insert sample data
cursor.executemany("""INSERT INTO type (id, name) VALUES (?, ?)""", [(1, 'Person'), (2, 'Integer')])
cursor.executemany("""INSERT INTO property (id, name, domain, range, required, multi) VALUES (?, ?, ?, ?, ?, ?)""", [(1, 'age', 1, 2, 0, None)])
cursor.executemany("""INSERT INTO property_type (id, type_id, property_id) VALUES (?, ?, ?)""", [(1, 1, 1), (2, 2, 1)])

connection.commit()

 # Example: Query and print data from the created tables
cursor.execute("SELECT * FROM type")
print("Type table:")
print(cursor.fetchall())

cursor.execute("SELECT * FROM property")
print("\nProperty table:")
print(cursor.fetchall())

cursor.execute("SELECT * FROM property_type")
print("\nProperty Type table:")
print(cursor.fetchall())

# Query to retrieve Person type with all its attributes including domain and range details
cursor.execute("""
    SELECT *
    FROM type t
    JOIN property_type pt ON t.id = pt.type_id
    JOIN property p ON pt.property_id = p.id
    LEFT JOIN type pd ON p.domain = pd.id
    LEFT JOIN type rd ON p.range = rd.id
    WHERE t.name = 'Person'
""")

# Fetch the results
results = cursor.fetchall()

# Convert results to JSON
data = {}
for row in results:
    if row[0] not in data:
        data[row[0]] = []
    data[row[0]].append({
        "property_name": row[1],
        "range": row[3] if row[3] is not None else "No range"
    })

# Print the JSON object
print(json.dumps(data, indent=4))

connection.close()

Type table:
[(1, 'Person'), (2, 'Integer')]

Property table:
[(1, 'age', 1, 2, 0, None)]

Property Type table:
[(1, 1, 1), (2, 2, 1)]
{
    "1": [
        {
            "property_name": "Person",
            "range": 1
        }
    ]
}
